# Trabalho 05 - Classificação

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np

In [3]:
df = pd.read_csv('data/trabalho5_dados_4.csv')
df

,Candidate ID,Name,Number of characters in Original Name,Month of Birth,Year of Birth,Gender,State (Location),10th percentage,12th percentage,10th Completion Year,...,Quantitative Ability 3,Quantitative Ability 4,Domain Skills 1,Domain Skills 2,Domain Test 3,Domain Test 4,Analytical Skills 1,Analytical Skills 2,Analytical Skills 3,Performance
0,1056,KnJ,11,JUL,Y7,A,S,74.20,69.60,Y14,...,74.301676,76.878613,48.2142857142857,61.151079,53.103448,89.781022,81.8181818181818,68.354430,88.505747,BP
1,1058,RvJ,12,JUN,Y7,A,Q,80.33,72.20,Y14,...,59.776536,60.693642,46.4285714285714,68.345324,58.620690,63.503650,89.0909090909091,88.607595,88.505747,BP
2,1059,PiG,13,JUL,Y8,B,S,77.80,74.40,Y14,...,68.715084,64.161850,44.6428571428571,64.028777,57.241379,64.963504,36.3636363636364,70.886076,19.540230,MP
3,1062,HpA,16,JAN,Y7,A,Q,77.50,72.40,Y14,...,67.597765,56.069364,64.2857142857143,76.978417,72.413793,75.182482,63.6363636363636,68.354430,63.218391,BP
4,1063,VaC,11,FEB,Y8,A,N,80.53,75.83,Y14,...,64.245810,60.693642,69.6428571428571,82.733813,58.620690,94.160584,49.0909090909091,60.759494,44.827586,BP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,85,LiJ,16,APR,Y8,A,A,78.60,88.58,Y14,...,64.245810,57.225434,53.5714285714286,84.172662,48.965517,76.642336,72.7272727272727,63.291139,79.310345,BP
463,474,DaG,9,OCT,Y6,B,J,89.00,71.00,Y13,...,58.659218,61.849711,MD,59.712230,58.620690,66.423358,87.2727272727273,81.012658,88.505747,BP
464,318,MaB,13,FEB,Y7,B,E,84.40,79.20,Y14,...,45.251397,50.289017,58.9285714285714,79.856115,73.793103,62.043796,49.0909090909091,75.949367,37.931034,MP
465,162,PrR,19,JUN,Y7,A,D,95.17,94.33,Y14,...,81.005587,86.127168,44.6428571428571,65.467626,48.965517,70.802920,81.8181818181818,88.607595,74.712644,BP
